# Import

In [1]:
import os
import copy
import json
import random
import os.path as osp
from glob import glob
from tqdm import tqdm

# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

from sklearn.model_selection import train_test_split

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
import mmcv
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
from mmcv import Config
print(get_compiling_cuda_version())
print(get_compiler_version())

from mmdet.datasets import build_dataset, CocoDataset
from mmdet.models import build_detector
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset
from mmdet.apis import train_detector, set_random_seed, init_detector, inference_detector

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(SEED)

1.7.0+cu101 True
2.7.0
10.1
GCC 7.3


# Data Preparation

In [2]:
# video_labels = pd.read_csv('/home/thinh/nfl/train_labels.csv').fillna(0)
# video_labels = video_labels[video_labels['frame'] != 0].reset_index(drop=True)
# # video_labels['impact'][(video_labels.confidence <= 1) | (video_labels.visibility <= 0)] = 0

# video_labels_with_impact = video_labels[video_labels['impact'] > 0]
# for row in tqdm(video_labels_with_impact[['video','frame','label']].values):
#     frames = np.array([-4, -3, -2, -1, 1, 2, 3, 4])+row[1]
#     video_labels.loc[(video_labels['video'] == row[0]) 
#                                  & (video_labels['frame'].isin(frames))
#                                  & (video_labels['label'] == row[2]), 'impact'] = 1

# video_labels['video_name'] = video_labels['video'].apply(lambda x: "_".join(x.split("_")[:2]))
# video_labels['image_name'] = video_labels['video'].str.replace('.mp4', '') + '_' + video_labels['frame'].astype(str) + '.png'
# video_labels = video_labels[video_labels.groupby('image_name')['impact'].transform("sum") > 0].reset_index(drop=True)
# video_labels.fillna({'impact': 2}, inplace=True)
# video_labels.head()

In [3]:
# # video_names = np.random.permutation(video_labels.video_name.unique())
# # valid_video_len = int(len(video_names)*0.2)
# # video_valid = video_names[:valid_video_len]
# # video_train = video_names[valid_video_len:]
# video_valid = ['57583_000082', '57586_004152', '57911_000147', '57997_003691', '57680_002206', '58095_004022', '57906_000718', '58005_001254', '57679_003316', '58103_003494', '57998_002181', '58048_000086']

# valid_df = video_labels[ video_labels.video_name.isin(video_valid)].reset_index(drop=True)
# train_df = video_labels[~video_labels.video_name.isin(video_valid)].reset_index(drop=True)

In [4]:
# with open('video_valid_44.txt', 'w') as writer:
#     writer.write(str(video_valid.tolist()))

# Generate COCO Format Json

In [5]:
def gen_classes(CLASSES):
    classes = list()      
    
    for i, CLASS in enumerate(CLASSES):
        single_class = {} 
        single_class['id'] = i + 1
        single_class['name'] = CLASS
        classes.append(single_class)
    return classes

def gen_objs(df, debug = False):
    
    if debug:
        
        df = df[:int(len(df)*0.05)]
        
    
    img_lists = list(df['image_name'].unique())
    imgs = list()
    objs = list()   
    
    ## gen images information
    for i in tqdm(range(len(img_lists))):
        '''
        
        I just notice that all images were preprocessed image size 720x1280
        If you want to check real image size, use this code below
        
        img = cv2.imread(os.path.join(data_path, img_lists[i]))
        
        single_img_obj = {}
        single_img_obj['file_name'] = img_lists[i]
        single_img_obj['height'] = img.shape[0]
        single_img_obj['width'] = img.shape[1]
        single_img_obj['id'] = i + 1
        '''
        
        single_img_obj = {}
        single_img_obj['file_name'] = img_lists[i]
        single_img_obj['height'] = 720 
        single_img_obj['width'] = 1280
        single_img_obj['id'] = i + 1        
        
        imgs.append(single_img_obj)
        
  
    ## gen objs information    
    for j in tqdm(range(len(df))):
        single_obj = {}
        single_obj['id'] = j + 1
        single_obj['image_id'] = img_lists.index(df['image_name'][j]) + 1
        single_obj['category_id'] = int(df['impact'][j] ) ## You need to customize if you want to add some 'impact' class
        single_obj['area'] = float(df['width'][j]*df['height'][j])
        single_obj['bbox'] = [int(df['left'][j]), int(df['top'][j]), int(df['width'][j]), int(df['height'][j])]
        single_obj['iscrowd'] = 0        
        
        objs.append(single_obj)
    
    print(f'images: {len(imgs)}, objs: {len(objs)}')
    
    return imgs, objs

def gen_coco(outpath, classes, objs, imgs, train=True):
    if train:
        data_dict = {}
        data_dict['images'] = []
        data_dict['annotations'] = []
        data_dict['categories'] = []
        data_dict['images'].extend(imgs)
        data_dict['annotations'].extend(objs)
        data_dict['categories'].extend(classes)
        
    else:
        data_dict = {}
        data_dict['images'] = []
        data_dict['categories'] = []
        
        data_dict['images'].extend(imgs)
        data_dict['categories'].extend(classes)   

    with open(outpath, 'w') as f_out:
        json.dump(data_dict, f_out)

In [6]:
CLASSES = ['impact', 'helmet']
classes = gen_classes(CLASSES)
classes

[{'id': 1, 'name': 'impact'}, {'id': 2, 'name': 'helmet'}]

In [7]:
# train_imgs, train_objs = gen_objs(train_df)
# valid_imgs, valid_objs = gen_objs(valid_df)

In [8]:
# gen_coco('train_44.json', classes, train_objs, train_imgs, train=True)
# gen_coco('valid_44.json', classes, valid_objs, valid_imgs, train=True)

# Dataset

In [9]:
@DATASETS.register_module()
class ImpactDataset(CocoDataset):
    CLASSES = set(CLASSES)

# Model build

In [10]:
config_file = 'cascade_rcnn_r50_fpn.py'
cfg = Config.fromfile(config_file)

In [11]:
cfg.total_epochs = 50
cfg.work_dir = '/home/thinh/nfl/mmdet-models/model_a1_44'
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.data.samples_per_gpu = 4

cfg.data_root = '/home/thinh/nfl/train_images/'

cfg.data.train.ann_file='train_44.json'
cfg.data.train.img_prefix= cfg.data_root

cfg.data.val.ann_file='valid_44.json',
cfg.data.val.img_prefix= cfg.data_root

cfg.data.test.ann_file='valid_44.json', ## I just set validation data for test because it's not our main purpose
cfg.data.test.img_prefix= cfg.data_root

In [12]:
print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='FasterRCNN',
    pretrained='torchvision://resnet50',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch'),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0),
        loss_bbox=dict(typ

# Train model

In [13]:
# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(
    cfg.model, train_cfg=cfg.train_cfg, test_cfg=cfg.test_cfg)
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

loading annotations into memory...
Done (t=0.61s)
creating index...
index created!


2021-01-03 14:34:27,023 - mmdet - INFO - load model from: torchvision://resnet50
2021-01-03 14:34:27,219 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

2021-01-03 14:34:30,625 - mmdet - INFO - load checkpoint from http://download.openmmlab.com/mmdetection/v2.0/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco/cascade_rcnn_r50_fpn_1x_coco_20200316-3dc56deb.pth


loading annotations into memory...
Done (t=0.08s)
creating index...
index created!


2021-01-03 14:34:30,767 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: roi_head.bbox_head.0.fc_cls.weight, roi_head.bbox_head.0.fc_cls.bias, roi_head.bbox_head.0.fc_reg.weight, roi_head.bbox_head.0.fc_reg.bias, roi_head.bbox_head.0.shared_fcs.0.weight, roi_head.bbox_head.0.shared_fcs.0.bias, roi_head.bbox_head.0.shared_fcs.1.weight, roi_head.bbox_head.0.shared_fcs.1.bias, roi_head.bbox_head.1.fc_cls.weight, roi_head.bbox_head.1.fc_cls.bias, roi_head.bbox_head.1.fc_reg.weight, roi_head.bbox_head.1.fc_reg.bias, roi_head.bbox_head.1.shared_fcs.0.weight, roi_head.bbox_head.1.shared_fcs.0.bias, roi_head.bbox_head.1.shared_fcs.1.weight, roi_head.bbox_head.1.shared_fcs.1.bias, roi_head.bbox_head.2.fc_cls.weight, roi_head.bbox_head.2.fc_cls.bias, roi_head.bbox_head.2.fc_reg.weight, roi_head.bbox_head.2.fc_reg.bias, roi_head.bbox_head.2.shared_fcs.0.weight, roi_head.bbox_head.2.shared_fcs.0.bias, roi_head.bbox_head.2.shared_fcs.1.w

2021-01-03 14:59:19,117 - mmdet - INFO - Epoch [1][1400/2001]	lr: 5.000e-03, eta: 1 day, 5:07:38, time: 1.057, data_time: 0.012, memory: 6189, loss_rpn_cls: 0.0110, loss_rpn_bbox: 0.0503, loss_cls: 0.2029, acc: 92.1191, loss_bbox: 0.3467, loss: 0.6109
2021-01-03 15:00:14,263 - mmdet - INFO - Epoch [1][1450/2001]	lr: 5.000e-03, eta: 1 day, 5:09:02, time: 1.103, data_time: 0.012, memory: 6189, loss_rpn_cls: 0.0099, loss_rpn_bbox: 0.0496, loss_cls: 0.1981, acc: 92.4023, loss_bbox: 0.3542, loss: 0.6118
2021-01-03 15:01:04,767 - mmdet - INFO - Epoch [1][1500/2001]	lr: 5.000e-03, eta: 1 day, 5:05:09, time: 1.010, data_time: 0.013, memory: 6189, loss_rpn_cls: 0.0097, loss_rpn_bbox: 0.0515, loss_cls: 0.2056, acc: 92.1641, loss_bbox: 0.3453, loss: 0.6120
2021-01-03 15:01:56,489 - mmdet - INFO - Epoch [1][1550/2001]	lr: 5.000e-03, eta: 1 day, 5:02:48, time: 1.035, data_time: 0.013, memory: 6189, loss_rpn_cls: 0.0118, loss_rpn_bbox: 0.0502, loss_cls: 0.2021, acc: 92.3457, loss_bbox: 0.3385, loss:

...

Loading and preparing results...
DONE (t=0.32s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=23.19s).
Accumulating evaluation results...


2021-01-04 15:09:27,830 - mmdet - INFO - Epoch(val) [29][2001]	0_bbox_mAP: 0.2850, 0_bbox_mAP_50: 0.4680, 0_bbox_mAP_75: 0.3210, 0_bbox_mAP_s: 0.2790, 0_bbox_mAP_m: 0.4630, 0_bbox_mAP_l: -1.0000, 0_bbox_mAP_copypaste: 0.285 0.468 0.321 0.279 0.463 -1.000


DONE (t=1.10s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.285
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.468
Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.321
Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.279
Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.463
Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = -1.000
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.353
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.353
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.353
Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.349
Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.512
Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = -1.000


2021-01-04 15:11:08,699 - mmdet - INFO - Epoch [30][50/2001]	lr: 5.000e-05, eta: 16:00:54, time: 2.011, data_time: 0.063, memory: 6189, loss_rpn_cls: 0.0023, loss_rpn_bbox: 0.0272, loss_cls: 0.0820, acc: 96.6182, loss_bbox: 0.2130, loss: 0.3245
2021-01-04 15:12:48,648 - mmdet - INFO - Epoch [30][100/2001]	lr: 5.000e-05, eta: 16:00:08, time: 1.994, data_time: 0.017, memory: 6189, loss_rpn_cls: 0.0020, loss_rpn_bbox: 0.0274, loss_cls: 0.0784, acc: 96.7373, loss_bbox: 0.2133, loss: 0.3212
2021-01-04 15:14:28,898 - mmdet - INFO - Epoch [30][150/2001]	lr: 5.000e-05, eta: 15:59:22, time: 2.005, data_time: 0.021, memory: 6189, loss_rpn_cls: 0.0023, loss_rpn_bbox: 0.0271, loss_cls: 0.0774, acc: 96.7959, loss_bbox: 0.2111, loss: 0.3180


KeyboardInterrupt: 